In [2]:
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/Object_Detection_Tensorflow_API.ipynb
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/Mask_RCNN/Mask_RCNN_Videos.ipynb

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
print(tf.__version__)

from utils import label_map_util

from utils import visualization_utils as vis_util

1.8.0


In [3]:

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90


In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [1]:
# Import moviepy module, you'll need to edit/save/watch video clips later on
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Imageio: 'ffmpeg-osx-v3.2.4' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-osx-v3.2.4 (33.8 MB)


Downloading: 8192/35458856 bytes (0.016384/35458856 bytes (0.0%40960/35458856 bytes (0.1%73728/35458856 bytes (0.2%90112/35458856 bytes (0.3%106496/35458856 bytes (0.3237568/35458856 bytes (0.7270336/35458856 bytes (0.8401408/35458856 bytes (1.1417792/35458856 bytes (1.2499712/35458856 bytes (1.4532480/35458856 bytes (1.5565248/35458856 bytes (1.6598016/35458856 bytes (1.7630784/35458856 bytes (1.8663552/35458856 bytes (1.9696320/35458856 bytes (2.0737280/35458856 bytes (2.1794624/35458856 bytes (2.2843776/35458856 bytes (2.4909312/35458856 bytes (2.6942080/35458856 bytes (2.7958464/35458856 bytes (2.7974848/35458856 bytes (2.7991232/35458856 bytes (2.81007616/35458856 bytes (2.8%1024000/35458856 bytes (2.9%1040384/35458856 bytes (2.9%1056768/35458856 bytes (3.0%1073152/35458856 bytes (3.0%1089536/35458856 bytes (3.1%1114112/35458856 bytes (3.1%1130496/35458856 bytes (3.2%1138688/35458856 bytes (3.2%1155072/35458856 bytes (3.3%1171456/35458856 bytes (3.3%1187840/35458856 bytes (3.3%120

5595136/35458856 bytes (15.85611520/35458856 bytes (15.85627904/35458856 bytes (15.95644288/35458856 bytes (15.95660672/35458856 bytes (16.05677056/35458856 bytes (16.05693440/35458856 bytes (16.15718016/35458856 bytes (16.15734400/35458856 bytes (16.25742592/35458856 bytes (16.25758976/35458856 bytes (16.25775360/35458856 bytes (16.35791744/35458856 bytes (16.35808128/35458856 bytes (16.45824512/35458856 bytes (16.45840896/35458856 bytes (16.55857280/35458856 bytes (16.55873664/35458856 bytes (16.65890048/35458856 bytes (16.65906432/35458856 bytes (16.75914624/35458856 bytes (16.75922816/35458856 bytes (16.75931008/35458856 bytes (16.75939200/35458856 bytes (16.75955584/35458856 bytes (16.85971968/35458856 bytes (16.85988352/35458856 bytes (16.96004736/35458856 bytes (16.96021120/35458856 bytes (17.06037504/35458856 bytes (17.06053888/35458856 bytes (17.16070272/35458856 bytes (17.16086656/35458856 bytes (17.26103040/35458856 bytes (17.26111232/35458856 bytes (17.26119424/35458856 byt

10444800/35458856 bytes (29.5%10461184/35458856 bytes (29.5%10477568/35458856 bytes (29.5%10493952/35458856 bytes (29.6%10510336/35458856 bytes (29.6%10534912/35458856 bytes (29.7%10551296/35458856 bytes (29.8%10575872/35458856 bytes (29.8%10592256/35458856 bytes (29.9%10608640/35458856 bytes (29.9%10625024/35458856 bytes (30.0%10641408/35458856 bytes (30.0%10657792/35458856 bytes (30.1%10690560/35458856 bytes (30.1%10723328/35458856 bytes (30.2%10731520/35458856 bytes (30.3%10739712/35458856 bytes (30.3%10756096/35458856 bytes (30.3%10772480/35458856 bytes (30.4%10788864/35458856 bytes (30.4%10805248/35458856 bytes (30.5%10821632/35458856 bytes (30.5%10838016/35458856 bytes (30.6%10854400/35458856 bytes (30.6%10870784/35458856 bytes (30.7%10887168/35458856 bytes (30.7%10903552/35458856 bytes (30.7%10911744/35458856 bytes (30.8%10928128/35458856 bytes (30.8%10936320/35458856 bytes (30.8%10952704/35458856 bytes (30.9%10969088/35458856 bytes (30.9%10985472/35458856 bytes (31.0%11001856/

15212544/35458856 bytes (42.9%15228928/35458856 bytes (42.9%15245312/35458856 bytes (43.0%15261696/35458856 bytes (43.0%15278080/35458856 bytes (43.1%15294464/35458856 bytes (43.1%15310848/35458856 bytes (43.2%15327232/35458856 bytes (43.2%15335424/35458856 bytes (43.2%15343616/35458856 bytes (43.3%15360000/35458856 bytes (43.3%15376384/35458856 bytes (43.4%15392768/35458856 bytes (43.4%15409152/35458856 bytes (43.5%15425536/35458856 bytes (43.5%15441920/35458856 bytes (43.5%15458304/35458856 bytes (43.6%15474688/35458856 bytes (43.6%15491072/35458856 bytes (43.7%15507456/35458856 bytes (43.7%15523840/35458856 bytes (43.8%15532032/35458856 bytes (43.8%15540224/35458856 bytes (43.8%15548416/35458856 bytes (43.8%15556608/35458856 bytes (43.9%15572992/35458856 bytes (43.9%15589376/35458856 bytes (44.0%15605760/35458856 bytes (44.0%15638528/35458856 bytes (44.1%15654912/35458856 bytes (44.1%15671296/35458856 bytes (44.2%15687680/35458856 bytes (44.2%15712256/35458856 bytes (44.3%15720448/

19365888/35458856 bytes (54.6%19374080/35458856 bytes (54.6%19382272/35458856 bytes (54.7%19390464/35458856 bytes (54.7%19406848/35458856 bytes (54.7%19423232/35458856 bytes (54.8%19439616/35458856 bytes (54.8%19456000/35458856 bytes (54.9%19472384/35458856 bytes (54.9%19488768/35458856 bytes (55.0%19505152/35458856 bytes (55.0%19521536/35458856 bytes (55.1%19537920/35458856 bytes (55.1%19554304/35458856 bytes (55.1%19562496/35458856 bytes (55.2%19570688/35458856 bytes (55.2%19578880/35458856 bytes (55.2%19587072/35458856 bytes (55.2%19603456/35458856 bytes (55.3%19619840/35458856 bytes (55.3%19636224/35458856 bytes (55.4%19652608/35458856 bytes (55.4%19668992/35458856 bytes (55.5%19685376/35458856 bytes (55.5%19701760/35458856 bytes (55.6%19718144/35458856 bytes (55.6%19734528/35458856 bytes (55.7%19750912/35458856 bytes (55.7%19767296/35458856 bytes (55.7%19775488/35458856 bytes (55.8%19783680/35458856 bytes (55.8%19800064/35458856 bytes (55.8%19816448/35458856 bytes (55.9%19832832/

24027136/35458856 bytes (67.8%24043520/35458856 bytes (67.8%24059904/35458856 bytes (67.9%24076288/35458856 bytes (67.9%24092672/35458856 bytes (67.9%24109056/35458856 bytes (68.0%24125440/35458856 bytes (68.0%24141824/35458856 bytes (68.1%24158208/35458856 bytes (68.1%24182784/35458856 bytes (68.2%24190976/35458856 bytes (68.2%24207360/35458856 bytes (68.3%24240128/35458856 bytes (68.4%24256512/35458856 bytes (68.4%24272896/35458856 bytes (68.5%24289280/35458856 bytes (68.5%24305664/35458856 bytes (68.5%24322048/35458856 bytes (68.6%24338432/35458856 bytes (68.6%24363008/35458856 bytes (68.7%24371200/35458856 bytes (68.7%24379392/35458856 bytes (68.8%24387584/35458856 bytes (68.8%24403968/35458856 bytes (68.8%24420352/35458856 bytes (68.9%24436736/35458856 bytes (68.9%24453120/35458856 bytes (69.0%24469504/35458856 bytes (69.0%24485888/35458856 bytes (69.1%24518656/35458856 bytes (69.1%24535040/35458856 bytes (69.2%24559616/35458856 bytes (69.3%24567808/35458856 bytes (69.3%24576000/

28532736/35458856 bytes (80.5%28549120/35458856 bytes (80.5%28581888/35458856 bytes (80.6%28606464/35458856 bytes (80.7%28622848/35458856 bytes (80.7%28631040/35458856 bytes (80.7%28647424/35458856 bytes (80.8%28663808/35458856 bytes (80.8%28680192/35458856 bytes (80.9%28696576/35458856 bytes (80.9%28729344/35458856 bytes (81.0%28745728/35458856 bytes (81.1%28762112/35458856 bytes (81.1%28786688/35458856 bytes (81.2%28794880/35458856 bytes (81.2%28827648/35458856 bytes (81.3%28844032/35458856 bytes (81.3%28860416/35458856 bytes (81.4%28876800/35458856 bytes (81.4%28893184/35458856 bytes (81.5%28909568/35458856 bytes (81.5%28925952/35458856 bytes (81.6%28958720/35458856 bytes (81.7%28975104/35458856 bytes (81.7%28983296/35458856 bytes (81.7%28991488/35458856 bytes (81.8%28999680/35458856 bytes (81.8%29007872/35458856 bytes (81.8%29024256/35458856 bytes (81.9%29040640/35458856 bytes (81.9%29057024/35458856 bytes (81.9%29089792/35458856 bytes (82.0%29106176/35458856 bytes (82.1%29122560/

33218560/35458856 bytes (93.7%33234944/35458856 bytes (93.7%33251328/35458856 bytes (93.8%33267712/35458856 bytes (93.8%33284096/35458856 bytes (93.9%33300480/35458856 bytes (93.9%33316864/35458856 bytes (94.0%33333248/35458856 bytes (94.0%33349632/35458856 bytes (94.1%33366016/35458856 bytes (94.1%33382400/35458856 bytes (94.1%33398784/35458856 bytes (94.2%33406976/35458856 bytes (94.2%33431552/35458856 bytes (94.3%33447936/35458856 bytes (94.3%33464320/35458856 bytes (94.4%33480704/35458856 bytes (94.4%33497088/35458856 bytes (94.5%33513472/35458856 bytes (94.5%33529856/35458856 bytes (94.6%33546240/35458856 bytes (94.6%33562624/35458856 bytes (94.7%33579008/35458856 bytes (94.7%33603584/35458856 bytes (94.8%33619968/35458856 bytes (94.8%33644544/35458856 bytes (94.9%33660928/35458856 bytes (94.9%33677312/35458856 bytes (95.0%33693696/35458856 bytes (95.0%33710080/35458856 bytes (95.1%33742848/35458856 bytes (95.2%33759232/35458856 bytes (95.2%33775616/35458856 bytes (95.3%33792000/

In [7]:
def detect_videos(image_np, sess, detection_graph):
    
    with detection_graph.as_default():
        
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
          ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
        if 'detection_masks' in tensor_dict:
            # The following processing is only for single image
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
            detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
            # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
            real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
            detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
            detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image_np.shape[0], image_np.shape[1])
            detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
            # Follow the convention by adding back the batch dimension
            tensor_dict['detection_masks'] = tf.expand_dims(
                detection_masks_reframed, 0)
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
        output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image_np, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
        if 'detection_masks' in output_dict:
            output_dict['detection_masks'] = output_dict['detection_masks'][0]

        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=1)
                
    return image_np

In [8]:
def process_image(image):  
    global counter
    if counter%1 ==0:
        with detection_graph.as_default():
            with tf.Session(graph=detection_graph) as sess:
                image_np = detect_videos(image, sess, detection_graph) 
    counter +=1 
    return image

In [16]:
input_filename = 'test_images/douyin3.mp4'
output_filename = 'test_images/douyin3-3.mp4'

counter = 0

# suclip(t_start, t_end)
clip_playing = VideoFileClip(input_filename).subclip(0,6.5) 

# NOTE: this function expects color images!!s
white_clip = clip_playing.fl_image(process_image) 

# write video to path with progress 
%time white_clip.write_videofile(output_filename)

[MoviePy] >>>> Building video test_images/douyin3-3.mp4
[MoviePy] Writing audio in douyin3-3TEMP_MPY_wvf_snd.mp3


100%|██████████| 144/144 [00:00<00:00, 562.92it/s]

[MoviePy] Done.
[MoviePy] Writing video test_images/douyin3-3.mp4




 79%|███████▉  | 155/196 [07:15<01:55,  2.81s/it]


 99%|█████████▉| 195/196 [09:15<00:02,  2.85s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_images/douyin3-3.mp4 

CPU times: user 8min 5s, sys: 1min 27s, total: 9min 33s
Wall time: 9min 16s
